## **Making New Layers and Models via Subclassing**

#### **Setup**

In [2]:
import tensorflow as tf
from tensorflow import keras

#### **The `Layer` Class: combination of state(weights) and computation**
###### **A layer encapsulates both a state(layer's weight) and a transformation from inputs to outputs. Let's have a look a densely-connected layer. It has a state: the variables `w` and `b`.**

In [3]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"),
            trainable=True
        )
    def call(self, inputs):
        return(tf.matmul(inputs, self.w) + self.b)

###### **We could use the layer by calling it on some tensor input(s):**

In [4]:
x = tf.ones((2,2))
linear_layer = Linear(4, 2)     # The column size(in this case 2), must be equal to the row size of x)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.04570365 -0.03962987 -0.03884789  0.09662069]
 [-0.04570365 -0.03962987 -0.03884789  0.09662069]], shape=(2, 4), dtype=float32)


###### **[NB] The weights `w` and `b` can be automatically tracked by the layer being set as layer attributes:**

In [5]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

###### **We can also add weights to a layer using the `add_weights()` method.**

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units),
            initializer="random_normal",
            trainable=True
        )
        self.b = self.add_weight(shape=(units,),
            initializer="zeros",
            trainable=True
        )
    def call(self, inputs):
        return(tf.matmul(inputs, self.w) + self.b)

x = tf.ones((2,2))
linear_layer = Linear(4, 2)     # The column size(in this case 2), must be equal to the row size of x)
y = linear_layer(x)
print(y)